# Do Inference on the Retrieval Dataset and save the predictions

In [ ]:
#! pip3 install torch==1.5.0 transformers==3.4.0
#! pip install pickle5
#! pip install datasets
#! pip install faiss-gpu cudatoolkit=10.0 -c pytorch

In [ ]:
import sys
import os
sys.path.append(os.path.dirname((os.path.abspath(''))))
 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, log_loss
from sklearn.model_selection import train_test_split
import pickle5 as pickle
from transformers import Trainer, TrainingArguments
from transformers import AutoModelForSequenceClassificatio
from src.models.predict_model import MAP_score
from src.data.utils import get_sentence_pairs
from src.models.train_text_encoder import Torch_dataset_mono, compute_metrics, predict_loop

np.random.seed(42)

#from google.colab import drive
#drive.mount('/content/drive')

In [ ]:
with open('../data/interim/europarl_en_it.pkl', "rb") as fh:
    original_sentence_pairs = pickle.load(fh)

feature_retrieval = pd.read_feather("../data/processed/feature_retrieval_en_it.feather")
feature_retrieval = feature_retrieval.rename(columns={"id_source": "source_id", "id_target": "target_id"})
dataset_pairs = get_sentence_pairs(original_sentence_pairs, feature_retrieval)

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained("../model/model_weighted/checkpoint-19000", num_labels=2)
training_args = TrainingArguments("model", per_device_eval_batch_size=32)
trainer = Trainer(
    args=training_args,
    model=model,                         # the instantiated 🤗 Transformers model to be trained
    #compute_metrics=compute_metrics
)


In [ ]:
predict_loop(trainer, dataset_pairs, "../model/predictions_weighted_en_it",save_steps=50000)